In [33]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import json

import pymongo
from pymongo import MongoClient

# Census & gmaps API Keys
# from config import (api_key, gkey)
# c = Census(api_key, year=2013)

# # Configure gmaps
# gmaps.configure(api_key=gkey)

In [34]:
# Making a Connection with MongoClient
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.Crimes_db

# Declare the collection
HighUnemp = db.HighUnemp
HighViolent = db.HighViolent
HighProperty = db.HighProperty
HighCrime = db.HighCrime
UnEmp10 = db.UnEmp10
ViolentTotal10 = db.ViolentTotal10
PropertyTotal10= db.PropertyTotal10
CrimeRate10 = db.CrimeRate10

In [35]:
statecsv= pd.read_csv("./data/statelatlong.csv")
statecsv.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [36]:
crimestatecsv= pd.read_csv("./data/crimebystatecombinedwithunemployment (1).csv")
crimestatecsv.head()

,state,unemployment,year,Population,violent total,Murder,rape,Robbery,Aggravated assault,property total,Burglary,Larceny theft,vehicle theft
0,AL,6.808,1976,3665000,388.8,15.1,21.7,96.0,256.0,3419.5,1170.0,1987.2,262.3
1,AL,7.325,1977,3690000,414.4,14.2,25.2,96.8,278.3,3298.2,1135.5,1881.9,280.7
2,AL,6.380,1978,3742000,419.1,13.3,25.5,99.1,281.2,3519.7,1229.3,1987.9,302.5
3,AL,7.158,1979,3769000,413.3,13.2,27.5,109.5,263.1,3830.5,1287.3,2223.2,320.1
4,AL,8.867,1980,3861466,448.5,13.2,30.0,132.1,273.2,4485.1,1526.7,2642.2,316.2


In [37]:
# Merge the datasets using the state columns, adding the latitude and longitude columns
crime_state_data = pd.merge(crimestatecsv,statecsv, how="left", left_on="state", right_on="State")

# Save the updated dataframe as a csv
crime_state_data.to_csv("./Unemp_crime_state_data.csv", encoding="utf-8", index=False)
crime_state_data.head()

,state,unemployment,year,Population,violent total,Murder,rape,Robbery,Aggravated assault,property total,Burglary,Larceny theft,vehicle theft,State,Latitude,Longitude,City
0,AL,6.808,1976,3665000,388.8,15.1,21.7,96.0,256.0,3419.5,1170.0,1987.2,262.3,AL,32.601011,-86.680736,Alabama
1,AL,7.325,1977,3690000,414.4,14.2,25.2,96.8,278.3,3298.2,1135.5,1881.9,280.7,AL,32.601011,-86.680736,Alabama
2,AL,6.380,1978,3742000,419.1,13.3,25.5,99.1,281.2,3519.7,1229.3,1987.9,302.5,AL,32.601011,-86.680736,Alabama
3,AL,7.158,1979,3769000,413.3,13.2,27.5,109.5,263.1,3830.5,1287.3,2223.2,320.1,AL,32.601011,-86.680736,Alabama
4,AL,8.867,1980,3861466,448.5,13.2,30.0,132.1,273.2,4485.1,1526.7,2642.2,316.2,AL,32.601011,-86.680736,Alabama


In [38]:
#filtering the data for 10 years from 2004 to 2014

newdf = crime_state_data[crime_state_data.year.isin([2004,2005,2006,2007,2008,2009, 2010,2011,2012,2013,2014])]
newdf.head()

,state,unemployment,year,Population,violent total,Murder,rape,Robbery,Aggravated assault,property total,Burglary,Larceny theft,vehicle theft,State,Latitude,Longitude,City
28,AL,5.683,2004,4525375,427.0,5.6,38.5,133.5,249.4,4029.3,987.0,2732.4,309.9,AL,32.601011,-86.680736,Alabama
29,AL,4.492,2005,4548327,433.0,8.2,34.4,141.7,248.3,3900.0,955.8,2656.0,289.0,AL,32.601011,-86.680736,Alabama
30,AL,4.033,2006,4599030,425.2,8.3,35.8,153.6,227.5,3941.0,973.7,2640.8,326.5,AL,32.601011,-86.680736,Alabama
31,AL,3.983,2007,4627851,448.9,8.9,33.4,159.9,246.7,3977.7,980.6,2689.5,307.7,AL,32.601011,-86.680736,Alabama
32,AL,5.658,2008,4661900,452.8,7.5,34.7,157.6,253.0,4084.5,1081.3,2714.3,288.9,AL,32.601011,-86.680736,Alabama


In [39]:
# new = old[['A','C','D']]
newdf2 = newdf[['state','unemployment','year','Population','Murder','rape','Robbery','Aggravated assault'
               ,'violent total','Burglary','Larceny theft','vehicle theft','property total',
               'Latitude','Longitude']]
newdf2

,state,unemployment,year,Population,Murder,rape,Robbery,Aggravated assault,violent total,Burglary,Larceny theft,vehicle theft,property total,Latitude,Longitude
28,AL,5.683,2004,4525375,5.6,38.5,133.5,249.4,427.0,987.0,2732.4,309.9,4029.3,32.601011,-86.680736
29,AL,4.492,2005,4548327,8.2,34.4,141.7,248.3,433.0,955.8,2656.0,289.0,3900.0,32.601011,-86.680736
30,AL,4.033,2006,4599030,8.3,35.8,153.6,227.5,425.2,973.7,2640.8,326.5,3941.0,32.601011,-86.680736
31,AL,3.983,2007,4627851,8.9,33.4,159.9,246.7,448.9,980.6,2689.5,307.7,3977.7,32.601011,-86.680736
32,AL,5.658,2008,4661900,7.5,34.7,157.6,253.0,452.8,1081.3,2714.3,288.9,4084.5,32.601011,-86.680736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,WY,6.458,2010,564554,1.4,28.7,13.6,154.1,197.9,381.0,1970.8,104.9,2456.6,43.000325,-107.554567
1985,WY,5.808,2011,567356,3.2,25.7,12.5,178.0,219.4,328.5,1849.5,91.8,2269.8,43.000325,-107.554567
1986,WY,5.317,2012,576626,2.4,26.7,10.6,161.6,201.3,368.5,1823.2,101.3,2293.0,43.000325,-107.554567
1987,WY,4.725,2013,583223,2.9,24.7,12.7,157.2,207.8,335.4,1761.8,99.1,2196.2,43.000325,-107.554567


In [40]:
newdf2.columns=['State','Unemployment','Year','Population','Murder','Rape','Robbery','Aggravated Assault'
               ,'Violent Total','Burglary','Larceny Theft','Vehicle Theft','Property Total',
               'Latitude','Longitude']
newdf2

,State,Unemployment,Year,Population,Murder,Rape,Robbery,Aggravated Assault,Violent Total,Burglary,Larceny Theft,Vehicle Theft,Property Total,Latitude,Longitude
28,AL,5.683,2004,4525375,5.6,38.5,133.5,249.4,427.0,987.0,2732.4,309.9,4029.3,32.601011,-86.680736
29,AL,4.492,2005,4548327,8.2,34.4,141.7,248.3,433.0,955.8,2656.0,289.0,3900.0,32.601011,-86.680736
30,AL,4.033,2006,4599030,8.3,35.8,153.6,227.5,425.2,973.7,2640.8,326.5,3941.0,32.601011,-86.680736
31,AL,3.983,2007,4627851,8.9,33.4,159.9,246.7,448.9,980.6,2689.5,307.7,3977.7,32.601011,-86.680736
32,AL,5.658,2008,4661900,7.5,34.7,157.6,253.0,452.8,1081.3,2714.3,288.9,4084.5,32.601011,-86.680736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,WY,6.458,2010,564554,1.4,28.7,13.6,154.1,197.9,381.0,1970.8,104.9,2456.6,43.000325,-107.554567
1985,WY,5.808,2011,567356,3.2,25.7,12.5,178.0,219.4,328.5,1849.5,91.8,2269.8,43.000325,-107.554567
1986,WY,5.317,2012,576626,2.4,26.7,10.6,161.6,201.3,368.5,1823.2,101.3,2293.0,43.000325,-107.554567
1987,WY,4.725,2013,583223,2.9,24.7,12.7,157.2,207.8,335.4,1761.8,99.1,2196.2,43.000325,-107.554567


In [47]:
newdf2=newdf2.dropna(how='any')

In [48]:
# Adding a new column for the Violent and Property Crime
newdf2['Total_CrimeRate'] = newdf2['Violent Total'] + newdf2['Property Total']
newdf2

,State,Unemployment,Year,Population,Murder,Rape,Robbery,Aggravated Assault,Violent Total,Burglary,Larceny Theft,Vehicle Theft,Property Total,Latitude,Longitude,Total_CrimeRate
28,AL,5.683,2004,4525375,5.6,38.5,133.5,249.4,427.0,987.0,2732.4,309.9,4029.3,32.601011,-86.680736,4456.3
29,AL,4.492,2005,4548327,8.2,34.4,141.7,248.3,433.0,955.8,2656.0,289.0,3900.0,32.601011,-86.680736,4333.0
30,AL,4.033,2006,4599030,8.3,35.8,153.6,227.5,425.2,973.7,2640.8,326.5,3941.0,32.601011,-86.680736,4366.2
31,AL,3.983,2007,4627851,8.9,33.4,159.9,246.7,448.9,980.6,2689.5,307.7,3977.7,32.601011,-86.680736,4426.6
32,AL,5.658,2008,4661900,7.5,34.7,157.6,253.0,452.8,1081.3,2714.3,288.9,4084.5,32.601011,-86.680736,4537.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,WY,6.458,2010,564554,1.4,28.7,13.6,154.1,197.9,381.0,1970.8,104.9,2456.6,43.000325,-107.554567,2654.5
1985,WY,5.808,2011,567356,3.2,25.7,12.5,178.0,219.4,328.5,1849.5,91.8,2269.8,43.000325,-107.554567,2489.2
1986,WY,5.317,2012,576626,2.4,26.7,10.6,161.6,201.3,368.5,1823.2,101.3,2293.0,43.000325,-107.554567,2494.3
1987,WY,4.725,2013,583223,2.9,24.7,12.7,157.2,207.8,335.4,1761.8,99.1,2196.2,43.000325,-107.554567,2404.0


In [49]:
# newdf_topUnemp_Year = newdf2['State','Unemployment','Year']

newdf_topUnemp_Year = newdf2.filter(['State','Unemployment','Year'], axis=1)
newdf_topUnemp_Year 

# print(newdf_topUnemp_Year.groupby('Year').
newdf_topUnemp_Year.groupby('Year')['Unemployment'].max().reset_index()

,Year,Unemployment
0,2004,7.783
1,2005,7.475
2,2006,7.000
3,2007,7.033
4,2008,7.983
5,2009,13.658
6,2010,13.525
7,2011,13.033
8,2012,11.167
9,2013,9.625


In [50]:
# Highest Unemployment rated states from 2004 - 2014  HighUnemp
newdf_topUnemp_Year = newdf2.filter(['State','Unemployment','Year'], axis=1)


newdf3=newdf_topUnemp_Year2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Unemployment", ascending=False)
.head(1)).droplevel(0).sort_values("Unemployment", ascending=False)

# newdf2[['State','Unemployment','Year']].groupby("Year", as_index=False).apply(lambda df:df.sort_values("Unemployment", ascending=False)
# .head(1)).droplevel(0).sort_values("Unemployment", ascending=False)

In [54]:
records = json.loads(newdf3.T.to_json()).values()
db.HighUnemp.insert_many(records)

# db.HighUnemp.insert_many(newdf3)

In [55]:
# Highest Violent Total for states from 2004 - 2014  HighViolent
newdf_topUnemp_Year = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Violent Total", ascending=False)
.head(1)).droplevel(0).sort_values("Violent Total", ascending=False)

In [56]:

# HighProperty = db.HighProperty
# HighCrime = db.HighCrime
# UnEmp10 = db.UnEmp10
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.HighViolent.insert_many(records)

In [59]:
# Highest Property Total for states from 2004 - 2014 - HighProperty
newdf3=newdf2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Property Total", ascending=False)
.head(1)).droplevel(0).sort_values("Property Total", ascending=False)

In [60]:
# HighCrime = db.HighCrime
# UnEmp10 = db.UnEmp10
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.HighProperty.insert_many(records)

In [61]:
# Highest CrimeRate (Violent+Property) Total for states from 2004 - 2014  - HighCrime
newdf3=newdf2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Total_CrimeRate", ascending=False)
.head(1)).droplevel(0).sort_values("Total_CrimeRate", ascending=False)

In [62]:

# UnEmp10 = db.UnEmp10
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.HighCrime.insert_many(records)

In [63]:
# 10 highest Unemployment rated states sorted for each year - UnEmp10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf_topUnemp_Year2.sort_values(by=['Year','Unemployment'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [64]:

# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.UnEmp10.insert_many(records)

In [29]:
# 10 highest Violent Totalstates sorted for each year - ViolentTotal10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.sort_values(by=['Year','Violent Total'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

,State,Unemployment,Year,Population,Murder,Rape,Robbery,Aggravated Assault,Violent Total,Burglary,Larceny Theft,Vehicle Theft,Property Total,Latitude,Longitude,Total_CrimeRate
350,DC,7.775,2014,658893,15.9,53.4,530.7,626.1,1244.4,526.0,4082.3,574.1,5182.5,38.899349,-77.014567,6426.9
77,AK,6.875,2014,736732,5.6,75.3,85.4,440.2,635.8,427.6,2096.4,236.0,2760.0,61.302501,-158.775020,3395.8
1130,NV,7.925,2014,2839099,6.0,35.0,209.7,372.1,635.6,772.3,1494.3,358.7,2625.4,38.502032,-117.023060,3261.0
1676,TN,6.583,2014,6549352,5.7,28.4,110.9,453.2,608.4,712.2,2156.0,192.4,3060.6,35.830521,-85.978599,3669.0
1247,NM,6.658,2014,2085572,4.8,51.4,100.0,421.8,597.4,887.3,2353.4,301.6,3542.3,34.166232,-106.026068,4139.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,NM,5.500,2004,1903006,8.9,54.6,108.4,515.6,687.4,1047.0,2736.1,415.2,4198.4,34.166232,-106.026068,4885.8
730,LA,5.925,2004,4506685,12.7,35.9,145.7,445.8,640.0,1006.5,2975.1,437.4,4419.1,30.973377,-91.429910,5059.1
67,AK,7.458,2004,657755,5.6,84.8,68.0,473.9,632.3,573.6,2456.7,340.6,3370.9,61.302501,-158.775020,4003.2
1120,NV,4.300,2004,2332898,7.4,40.9,210.3,357.8,616.4,992.0,2247.8,970.3,4210.0,38.502032,-117.023060,4826.4


In [65]:
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.ViolentTotal10.insert_many(records)

In [66]:
# 10 highest Property Total for states sorted for each year -PropertyTotal10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.sort_values(by=['Year','Property Total'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [67]:
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.PropertyTotal10.insert_many(records)

In [68]:
# 10 highest Total Crime for states sorted for each year -CrimeRate10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.sort_values(by=['Year','Total_CrimeRate'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [69]:
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.CrimeRate10.insert_many(records)

In [15]:
# converting the dataframe to json object and saved into Export_DataFrame.json file.
# js_all = 
newdf2.to_json(r'Export_DataFrame.json',orient = 'records')
# print(js_all)

In [16]:
# Function to convert the dataframe to geojson data.

def df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [17]:
cols = ['State','Unemployment','Year','Population','Murder','Rape','Robbery','Aggravated Assault'
               ,'Violent Total','Burglary','Larceny Theft','Vehicle Theft','Property Total','Total_CrimeRate']
# geojson = df_to_geojson(df_geo, cols)

# calling the function to convert the dataframe to geojson data.

geojson = df_to_geojson(newdf2, cols)

In [18]:
# save the geojson result to a file  'dataset.js'
output_filename = 'dataset.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson)))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson['features'])))

561 geotagged features saved to file
